In [1]:
cd ..

/home/ubuntu/Code/experiments-motion


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
sys.path.append('./Motion')

In [4]:
import torch
import numpy as np
import pandas as pd
import yaml
from torch.utils.data import DataLoader
from tqdm import tqdm
from IPython.display import display, HTML

In [5]:
from motion import Quaternion
from h36m.skeleton import H36MSkeleton
from h36m.dataset.h36m_torch_dataset import H36MTorchDataset
from h36m.dataset.h36m_dataset import H36MDataset
from h36m.dataset.h36m_test_dataset import H36MTestDataset

In [6]:
from metrics import MeanAngleL2Error, MeanPerJointPositionError, NegativeLogLikelihood

# Load H3.6M Skeleton

In [7]:
# Load skeleton configuration
with open('./config/h36m_skeleton.yaml', 'r') as stream:
    skeleton = H36MSkeleton(**yaml.safe_load(stream))

## Define Eval Metrics

In [8]:
mae_l2_metric = MeanAngleL2Error(ignore_root=True) # We ignore the root rotation in the world for the origin joint
mpjpe_metric = MeanPerJointPositionError()
nll_metric = NegativeLogLikelihood()

In [9]:
t = [0.08, 0.16, 0.32, 0.4, 0.56, 0.72, 0.88, 1.]

## Load Evaluation Data

In [10]:
DATA_PATH = './data/processed/h3.6m.npz'

In [11]:
h36m_dataset = H36MDataset(DATA_PATH, dataset_fps=50, dataset_downsample_factor=2)

In [12]:
dataset_eval = H36MTorchDataset(h36m_dataset,
                                subjects=['S5'],
                                history_length=50,
                                prediction_horizon=25)

# Evaluation of History Repeats itself on H3.6M dataset

In [13]:
sys.path.append('./compare/HisRepItself/')
from model import AttModel

### Load the Model

In [14]:
in_features = 48
d_model = 256
num_stage = 12
kernel_size = 10
dct_n = 20
input_n = 50
output_n = 25
model = AttModel.AttModel(in_features=in_features, kernel_size=kernel_size, d_model=d_model,
                             num_stage=num_stage, dct_n=dct_n)

In [17]:
ckpt = torch.load('./compare/HisRepItself/checkpoint/pretrained/h36m_ang_in50_out10_dctn20/ckpt_best.pth.tar', map_location=torch.device('cpu'))

In [18]:
model.load_state_dict(ckpt['state_dict'])
model.eval()
print('Model Loaded')

Model Loaded


In [19]:
dim_used = np.array([6, 7, 8, 9, 12, 13, 14, 15, 21, 22, 23, 24, 27, 28, 29, 30, 36, 37, 38, 39, 40, 41, 42,
                         43, 44, 45, 46, 47, 51, 52, 53, 54, 55, 56, 57, 60, 61, 62, 75, 76, 77, 78, 79, 80, 81, 84, 85,
                         86])

## Evaluate HistRepItself on 8 Series per Action on Subject 5

In [20]:
ACTIONS = ['walking', 'eating', 'smoking', 'discussion', 'directions', 'greeting', 'phoning', 'posing',
           'purchases', 'sitting', 'sittingdown', 'takingphoto', 'waiting', 'walkingdog', 'walkingtogether',]

In [21]:
result = pd.DataFrame(columns=pd.MultiIndex.from_product([['Mean Angle Error (L2)', 'Mean per Joint Position Error'], ['average'] + ACTIONS]))
with torch.no_grad():
    for action in tqdm(ACTIONS):
        mae_l2_metric.reset()
        mpjpe_metric.reset()
        dataset_test_action = H36MTestDataset(h36m_dataset,
                                              action=action,
                                              subjects=['S5'],
                                              num_samples=8,
                                              history_length=50,
                                              prediction_horizon=25)
        
        for x, y in DataLoader(dataset_test_action, batch_size=256):
            mpjpe_metric.reset()
            itera = 3
            x_input = torch.cat([x, torch.zeros_like(y)], dim=1)
        
            x_expmap = Quaternion(x_input).axis_angle.flatten(start_dim=-2)
            x_expmap = torch.cat([torch.zeros(x_expmap.shape[:-1] +(3,)), x_expmap], dim=-1)

            # Remove static nodes
            x_dim = -x_expmap[..., dim_used]

            # Run Model 
            y_pred = model(x_dim, output_n=10, itera=itera, input_n=input_n)
            y_pred = -y_pred[:, kernel_size:].transpose(1, 2).reshape([x.shape[0], 10 * itera, -1])[:, :output_n]

            # Add static rotations
            y_full = torch.zeros(y.shape[:-2] + x_expmap.shape[-1:])
            y_full[..., dim_used] = y_pred
            y_final = y_full.view(y_full.shape[:-1] + (-1, 3))[..., 1:, :]

            y_final_q = Quaternion(angle=torch.norm(y_final, dim=-1), axis=y_final).q
            
            mae_l2_metric.update((y_final_q, y))
        
            pos_final = skeleton(y_final_q.view(-1, 32, 4)).view(-1, 25, 32, 3)
            pos_y = skeleton(y.view(-1, 32, 4)).view(-1, 25, 32, 3)
            mpjpe_metric.update((pos_final, pos_y))

        mae_l2 = mae_l2_metric.compute().numpy()
        mpjpe = mpjpe_metric.compute().numpy()
        result['Mean Angle Error (L2)', action] = pd.Series(data=mae_l2, index=0.04*np.arange(1, mae_l2.shape[0] + 1))
        result['Mean per Joint Position Error', action] = pd.Series(data=mpjpe, index=0.04*np.arange(1, mpjpe.shape[0] + 1))
result['Mean Angle Error (L2)', 'average'] = result['Mean Angle Error (L2)'].mean(axis=1)
result['Mean per Joint Position Error', 'average'] = result['Mean per Joint Position Error'].mean(axis=1)

100%|██████████| 15/15 [00:05<00:00,  2.82it/s]


### Mean Angle Error (L2)

In [22]:
display(HTML(result['Mean Angle Error (L2)'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,0.27,0.18,0.16,0.22,0.20,0.25,0.35,0.53,0.19,0.42,0.29,0.30,0.16,0.22,0.46,0.14
0.16,0.52,0.30,0.29,0.42,0.52,0.43,0.60,1.01,0.46,0.65,0.47,0.63,0.36,0.49,0.78,0.32
0.32,0.82,0.46,0.49,0.86,0.78,0.60,0.95,1.22,1.09,1.00,0.83,0.92,0.58,0.92,1.05,0.50
0.40,0.93,0.51,0.60,0.80,0.87,0.69,1.14,1.29,1.35,1.07,1.01,1.04,0.70,1.14,1.23,0.55
0.56,1.14,0.59,0.74,0.86,1.30,0.81,1.48,1.42,1.59,1.43,1.16,1.18,0.83,1.54,1.58,0.63
0.72,1.28,0.62,0.81,1.00,1.54,1.03,1.47,1.55,1.83,1.53,1.29,1.42,0.91,1.92,1.65,0.68
0.88,1.48,0.61,1.01,1.34,1.66,1.25,1.61,1.68,2.14,1.94,1.50,1.55,1.01,2.25,1.79,0.80
1.00,1.59,0.64,1.11,1.58,1.72,1.29,1.57,1.68,2.34,2.24,1.55,1.69,1.08,2.33,1.85,1.18


In [23]:
display(HTML(result['Mean per Joint Position Error'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,13.50,11.33,9.21,8.39,10.47,10.91,15.13,13.64,10.78,21.17,12.64,12.05,8.03,12.16,35.35,11.17
0.16,27.67,19.59,19.30,15.81,24.57,21.24,29.41,23.52,27.36,43.67,29.67,26.24,16.23,29.05,64.97,24.47
0.32,54.89,30.80,38.83,26.83,47.25,45.57,72.02,44.05,74.04,67.97,60.02,53.55,41.02,71.29,105.78,44.35
0.40,66.37,37.78,47.91,30.60,54.80,56.57,89.46,49.97,92.71,78.36,70.89,64.50,52.26,90.54,127.83,51.36
0.56,83.87,39.49,61.11,34.23,79.76,78.11,106.03,66.25,117.12,100.20,89.82,82.70,73.06,118.74,149.65,61.85
0.72,96.57,43.05,67.11,40.35,101.36,96.99,106.94,83.26,142.01,118.92,106.93,103.08,79.90,131.95,161.54,65.08
0.88,110.21,51.13,77.07,53.41,110.70,110.15,104.78,102.29,179.49,125.69,117.66,119.66,86.87,158.07,176.65,79.46
1.00,118.21,52.97,77.65,60.27,107.49,111.55,97.55,120.62,216.51,127.01,120.75,138.55,90.44,181.49,183.97,86.27


## Evaluate HistRepItself on 256 Series per Action on Subject 5

In [22]:
ACTIONS = ['walking', 'eating', 'smoking', 'discussion', 'directions', 'greeting', 'phoning', 'posing',
           'purchases', 'sitting', 'sittingdown', 'takingphoto', 'waiting', 'walkingdog', 'walkingtogether',]

In [23]:
result = pd.DataFrame(columns=pd.MultiIndex.from_product([['Mean Angle Error (L2)', 'Mean per Joint Position Error'], ['average'] + ACTIONS]))
with torch.no_grad():
    for action in tqdm(ACTIONS):
        mae_l2_metric.reset()
        mpjpe_metric.reset()
        dataset_test_action = H36MTestDataset(h36m_dataset,
                                              action=action,
                                              subjects=['S5'],
                                              num_samples=256,
                                              history_length=50,
                                              prediction_horizon=25)
        
        for x, y in DataLoader(dataset_test_action, batch_size=256):
            mpjpe_metric.reset()
            itera = 3
            x_input = torch.cat([x, torch.zeros_like(y)], dim=1)
        
            x_expmap = Quaternion(x_input).axis_angle.flatten(start_dim=-2)
            x_expmap = torch.cat([torch.zeros(x_expmap.shape[:-1] +(3,)), x_expmap], dim=-1)

            # Remove static nodes
            x_dim = -x_expmap[..., dim_used]

            # Run Model 
            y_pred = model(x_dim, output_n=10, itera=itera, input_n=input_n)
            y_pred = -y_pred[:, kernel_size:].transpose(1, 2).reshape([x.shape[0], 10 * itera, -1])[:, :output_n]

            # Add static rotations
            y_full = torch.zeros(y.shape[:-2] + x_expmap.shape[-1:])
            y_full[..., dim_used] = y_pred
            y_final = y_full.view(y_full.shape[:-1] + (-1, 3))[..., 1:, :]

            y_final_q = Quaternion(angle=torch.norm(y_final, dim=-1), axis=y_final).q
            
            mae_l2_metric.update((y_final_q, y))
        
            pos_final = skeleton(y_final_q.view(-1, 32, 4)).view(-1, 25, 32, 3)
            pos_y = skeleton(y.view(-1, 32, 4)).view(-1, 25, 32, 3)
            mpjpe_metric.update((pos_final, pos_y))

        mae_l2 = mae_l2_metric.compute().numpy()
        mpjpe = mpjpe_metric.compute().numpy()
        result['Mean Angle Error (L2)', action] = pd.Series(data=mae_l2, index=0.04*np.arange(1, mae_l2.shape[0] + 1))
        result['Mean per Joint Position Error', action] = pd.Series(data=mpjpe, index=0.04*np.arange(1, mpjpe.shape[0] + 1))
result['Mean Angle Error (L2)', 'average'] = result['Mean Angle Error (L2)'].mean(axis=1)
result['Mean per Joint Position Error', 'average'] = result['Mean per Joint Position Error'].mean(axis=1)

100%|██████████| 15/15 [01:22<00:00,  5.53s/it]


### Mean Angle Error (L2)

In [24]:
display(HTML(result['Mean Angle Error (L2)'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,0.28,0.24,0.18,0.21,0.31,0.19,0.39,0.29,0.27,0.43,0.25,0.41,0.19,0.25,0.41,0.21
0.16,0.52,0.43,0.37,0.38,0.61,0.38,0.71,0.52,0.55,0.78,0.49,0.72,0.34,0.46,0.68,0.38
0.32,0.88,0.66,0.67,0.65,1.02,0.74,1.17,0.91,1.00,1.21,0.91,1.17,0.60,0.88,1.01,0.62
0.40,1.02,0.71,0.79,0.79,1.17,0.90,1.35,1.05,1.21,1.31,1.06,1.36,0.72,1.05,1.12,0.71
0.56,1.23,0.84,0.95,0.99,1.44,1.08,1.60,1.24,1.54,1.47,1.33,1.66,0.92,1.28,1.30,0.86
0.72,1.40,0.91,1.11,1.15,1.57,1.22,1.78,1.47,1.80,1.62,1.59,1.88,1.07,1.47,1.45,0.94
0.88,1.55,0.99,1.23,1.30,1.68,1.35,1.93,1.63,2.10,1.74,1.78,2.11,1.21,1.63,1.54,1.00
1.00,1.64,1.03,1.30,1.42,1.76,1.42,1.99,1.72,2.24,1.81,1.88,2.20,1.33,1.75,1.63,1.04


In [25]:
display(HTML(result['Mean per Joint Position Error'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,12.38,12.63,7.64,8.58,12.01,8.91,16.60,10.43,12.83,15.12,10.99,16.11,9.96,10.40,22.54,10.89
0.16,25.87,24.38,16.23,17.24,26.36,20.82,35.76,20.81,29.28,33.06,22.95,32.46,20.64,22.40,43.87,21.80
0.32,51.42,41.26,31.78,33.17,56.19,47.15,72.73,41.69,66.46,64.79,47.17,60.82,43.47,48.26,76.01,40.36
0.40,62.47,46.07,39.10,40.41,69.64,58.73,87.89,51.27,83.74,77.74,58.58,73.89,54.20,59.80,88.98,47.00
0.56,81.43,53.30,52.83,52.48,92.21,75.81,112.96,68.81,114.87,98.78,79.02,99.00,74.41,79.70,110.28,57.00
0.72,96.25,58.41,64.09,61.97,109.10,91.20,129.09,84.19,142.99,116.41,96.28,119.21,92.50,94.78,122.20,61.36
0.88,108.57,62.62,73.40,69.59,120.48,103.52,141.64,98.31,166.41,131.07,110.39,135.01,107.47,106.28,136.22,66.17
1.00,116.38,65.42,78.65,74.72,125.98,110.12,146.81,107.44,181.71,139.97,118.77,146.01,117.73,113.93,148.21,70.30


## Evaluate MPJPE Version of HistRepItself

### Load the Model

In [26]:
in_features = 66
d_model = 256
num_stage = 12
kernel_size = 10
dct_n = 20
input_n = 50
output_n = 25
model = AttModel.AttModel(in_features=in_features, kernel_size=kernel_size, d_model=d_model,
                             num_stage=num_stage, dct_n=dct_n)

In [27]:
ckpt = torch.load('./h36m/comparison/HisRepItself/checkpoint/pretrained/h36m_3d_in50_out10_dctn20/ckpt_best.pth.tar', map_location=torch.device('cpu'))

In [28]:
model.load_state_dict(ckpt['state_dict'])
model.eval()
print('Model Loaded')

Model Loaded


In [29]:
dim_used = np.array([6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 21, 22, 23, 24, 25,
                         26, 27, 28, 29, 30, 31, 32, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                         46, 47, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 64, 65, 66, 67, 68,
                         75, 76, 77, 78, 79, 80, 81, 82, 83, 87, 88, 89, 90, 91, 92])

In [30]:
joint_to_ignore = np.array([16, 20, 23, 24, 28, 31])
index_to_ignore = np.concatenate((joint_to_ignore * 3, joint_to_ignore * 3 + 1, joint_to_ignore * 3 + 2))
joint_equal = np.array([13, 19, 22, 13, 27, 30])
index_to_equal = np.concatenate((joint_equal * 3, joint_equal * 3 + 1, joint_equal * 3 + 2))

## Evaluate on 256 Series per Action on Subject 5

In [31]:
ACTIONS = ['walking', 'eating', 'smoking', 'discussion', 'directions', 'greeting', 'phoning', 'posing',
           'purchases', 'sitting', 'sittingdown', 'takingphoto', 'waiting', 'walkingdog', 'walkingtogether',]

In [33]:
result = pd.DataFrame(columns=pd.MultiIndex.from_product([['Mean Angle Error (L2)', 'Mean per Joint Position Error'], ['average'] + ACTIONS]))
with torch.no_grad():
    for action in tqdm(ACTIONS):
        mae_l2_metric.reset()
        mpjpe_metric.reset()
        dataset_test_action = H36MTestDataset(h36m_dataset,
                                              action=action,
                                              subjects=['S5'],
                                              num_samples=256,
                                              history_length=50,
                                              prediction_horizon=25)
        
        for x, y in DataLoader(dataset_test_action, batch_size=256):
            mpjpe_metric.reset()
            itera = 3
            x_pos = skeleton(x.view(-1, 32, 4)).view(-1, 50, 32, 3)
            y_pos = skeleton(y.view(-1, 32, 4)).view(-1, 25, 32, 3)
            x_input = torch.cat([x_pos, torch.zeros_like(y_pos)], dim=1).flatten(start_dim=-2)

            # Remove static nodes
            x_dim = x_input[..., dim_used]

            # Run Model 
            y_pred = model(x_dim, output_n=10, itera=itera, input_n=input_n)
    
            y_pred = y_pred[:, kernel_size:].transpose(1, 2).reshape([x.shape[0], 10 * itera, -1])[:, :output_n]
            
            # Add static positions
            y_full = x_input[:, [0]*25]
            y_full[..., dim_used] = y_pred
            y_full[..., index_to_ignore] = y_full[..., index_to_equal]
            y_final = y_full.view(y_full.shape[:-1] + (-1, 3))

            mpjpe_metric.update((y_final, y_pos))
            
        mpjpe = mpjpe_metric.compute().numpy()
        result['Mean per Joint Position Error', action] = pd.Series(data=mpjpe, index=0.04*np.arange(1, mpjpe.shape[0] + 1))
result['Mean per Joint Position Error', 'average'] = result['Mean per Joint Position Error'].mean(axis=1)

100%|██████████| 15/15 [01:54<00:00,  7.62s/it]


### Mean per Joint Position Error

In [34]:
display(HTML(result['Mean per Joint Position Error'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,10.45,10.05,6.40,7.04,10.17,7.37,13.70,8.62,10.16,13.01,9.30,14.89,8.32,8.73,20.07,8.86
0.16,22.59,19.53,13.99,14.91,23.37,18.40,30.11,18.26,24.17,29.16,20.06,30.69,18.39,19.16,40.27,18.45
0.32,47.14,34.23,28.67,29.92,52.14,44.52,63.84,38.99,58.49,60.38,44.29,59.10,40.69,43.41,73.33,35.09
0.40,58.26,39.80,36.15,36.43,65.42,56.53,78.13,49.17,75.80,73.92,55.99,72.03,51.47,54.92,86.29,41.85
0.56,77.26,47.38,49.96,47.60,86.63,73.88,101.92,67.42,107.55,95.56,76.44,96.99,72.09,74.51,108.22,52.67
0.72,91.76,52.06,61.38,56.62,102.19,88.18,118.44,82.90,136.78,110.90,93.06,116.12,90.40,88.98,120.63,57.76
0.88,104.13,55.50,70.62,64.38,113.18,100.10,132.67,96.50,161.40,125.02,106.97,132.08,105.47,100.27,135.86,61.99
1.00,112.08,58.13,75.65,69.48,119.80,106.49,138.82,104.98,178.20,134.22,115.94,143.55,115.93,108.19,146.85,64.90


## Evaluate Zero Velocity MAE (L2) on 8 Series per Action on Subject 5

In [14]:
class ZeroVelocity(torch.nn.Module):
    def forward(self, q, ph=1):
        return q[:, [-1] * ph]
model = ZeroVelocity()

In [15]:
ACTIONS = ['walking', 'eating', 'smoking', 'discussion', 'directions', 'greeting', 'phoning', 'posing',
           'purchases', 'sitting', 'sittingdown', 'takingphoto', 'waiting', 'walkingdog', 'walkingtogether',]

In [16]:
result = pd.DataFrame(columns=pd.MultiIndex.from_product([['Mean Angle Error (L2)', 'Mean per Joint Position Error'], ['average'] + ACTIONS]))
with torch.no_grad():
    for action in tqdm(ACTIONS):
        mae_l2_metric.reset()
        mpjpe_metric.reset()
        dataset_test_action = H36MTestDataset(h36m_dataset,
                                              action=action,
                                              subjects=['S5'],
                                              num_samples=8,
                                              history_length=50,
                                              prediction_horizon=25)
        
        for x, y in DataLoader(dataset_test_action, batch_size=128):
            # Run Model 
            y_pred = model(x, 25)

            # Add static rotations
            
            mae_l2_metric.update((y_pred, y))
        
            pos_final = skeleton(y_pred.view(-1, 32, 4)).view(-1, 25, 32, 3)
            pos_y = skeleton(y.view(-1, 32, 4)).view(-1, 25, 32, 3)
            mpjpe_metric.update((pos_final, pos_y))

        mae_l2 = mae_l2_metric.compute().numpy()
        mpjpe = mpjpe_metric.compute().numpy()
        result['Mean Angle Error (L2)', action] = pd.Series(data=mae_l2, index=0.04*np.arange(1, mae_l2.shape[0] + 1))
        result['Mean per Joint Position Error', action] = pd.Series(data=mpjpe, index=0.04*np.arange(1, mpjpe.shape[0] + 1))
result['Mean Angle Error (L2)', 'average'] = result['Mean Angle Error (L2)'].mean(axis=1)
result['Mean per Joint Position Error', 'average'] = result['Mean per Joint Position Error'].mean(axis=1)

100%|██████████| 15/15 [00:00<00:00, 21.97it/s]


### Mean Angle Error (L2)

In [17]:
display(HTML(result['Mean Angle Error (L2)'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,0.40,0.39,0.27,0.26,0.31,0.39,0.54,0.64,0.28,0.62,0.40,0.39,0.25,0.34,0.60,0.33
0.16,0.71,0.68,0.48,0.48,0.67,0.59,0.89,1.21,0.57,0.88,0.63,0.74,0.51,0.67,0.98,0.66
0.32,1.07,0.99,0.73,0.97,0.94,0.79,1.30,1.57,1.13,1.19,1.02,1.07,0.79,1.22,1.36,0.94
0.40,1.20,1.15,0.86,0.95,1.04,0.89,1.49,1.70,1.37,1.27,1.18,1.19,0.92,1.47,1.50,0.99
0.56,1.42,1.35,1.04,1.02,1.41,1.02,1.79,1.81,1.81,1.64,1.26,1.36,1.03,1.89,1.74,1.10
0.72,1.57,1.37,1.10,1.14,1.71,1.22,1.77,1.94,2.23,1.62,1.36,1.57,1.13,2.27,1.87,1.22
0.88,1.75,1.34,1.27,1.47,1.86,1.47,1.85,2.05,2.58,2.09,1.57,1.70,1.22,2.57,1.95,1.22
1.00,1.85,1.32,1.38,1.69,1.96,1.50,1.80,2.04,2.78,2.45,1.63,1.80,1.27,2.63,1.96,1.52


In [18]:
display(HTML(result['Mean per Joint Position Error'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,26.02,32.84,18.69,12.76,23.83,25.64,29.37,25.60,17.28,32.01,23.84,20.65,16.26,26.73,56.76,28.09
0.16,47.58,58.09,36.40,23.41,45.29,41.27,56.23,46.08,34.96,57.41,48.75,36.69,30.19,50.60,94.92,53.39
0.32,82.13,98.04,68.30,36.53,70.86,57.91,109.42,82.29,76.99,85.62,85.69,69.30,58.14,104.56,141.39,86.96
0.40,95.65,114.46,87.09,40.34,77.99,70.26,126.81,93.56,93.35,94.35,96.56,81.99,70.59,126.34,160.64,100.34
0.56,113.78,128.71,116.47,42.65,101.84,90.70,131.08,116.30,116.41,111.03,108.85,99.80,89.34,154.18,186.03,113.30
0.72,125.92,140.19,128.34,51.67,121.70,111.85,120.59,138.91,146.15,127.21,122.92,119.21,92.89,158.19,193.09,115.94
0.88,135.41,152.79,133.41,65.52,128.49,126.18,113.06,154.15,189.53,131.56,131.97,135.57,94.69,168.75,181.89,123.60
1.00,140.36,148.43,121.49,71.60,123.48,126.73,104.35,163.84,229.67,135.99,134.08,153.33,96.80,180.97,181.46,133.12


## Evaluate Zero Velocity MAE (L2) on 256 Series per Action on Subject 5

In [134]:
result = pd.DataFrame(columns=pd.MultiIndex.from_product([['Mean Angle Error (L2)', 'Mean per Joint Position Error'], ['average'] + ACTIONS]))
with torch.no_grad():
    for action in tqdm(ACTIONS):
        mae_l2_metric.reset()
        mpjpe_metric.reset()
        dataset_test_action = H36MTestDataset(h36m_dataset,
                                              action=action,
                                              subjects=['S5'],
                                              num_samples=256,
                                              history_length=50,
                                              prediction_horizon=25)
        
        for x, y in DataLoader(dataset_test_action, batch_size=128):
            # Run Model 
            y_pred = model(x, 25)

            # Add static rotations
            
            mae_l2_metric.update((y_pred, y))
        
            pos_final = skeleton(y_pred.view(-1, 32, 4)).view(-1, 25, 32, 3)
            pos_y = skeleton(y.view(-1, 32, 4)).view(-1, 25, 32, 3)
            mpjpe_metric.update((pos_final, pos_y))

        mae_l2 = mae_l2_metric.compute().numpy()
        mpjpe = mpjpe_metric.compute().numpy()
        result['Mean Angle Error (L2)', action] = pd.Series(data=mae_l2, index=0.04*np.arange(1, mae_l2.shape[0] + 1))
        result['Mean per Joint Position Error', action] = pd.Series(data=mpjpe, index=0.04*np.arange(1, mpjpe.shape[0] + 1))
result['Mean Angle Error (L2)', 'average'] = result['Mean Angle Error (L2)'].mean(axis=1)
result['Mean per Joint Position Error', 'average'] = result['Mean per Joint Position Error'].mean(axis=1)

100%|██████████| 15/15 [00:08<00:00,  1.81it/s]


### Mean Angle Error (L2)

In [135]:
display(HTML(result['Mean Angle Error (L2)'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,0.40,0.43,0.28,0.30,0.46,0.30,0.54,0.39,0.40,0.57,0.33,0.50,0.26,0.37,0.53,0.37
0.16,0.70,0.78,0.53,0.53,0.80,0.54,0.91,0.69,0.74,0.96,0.60,0.84,0.44,0.64,0.85,0.66
0.32,1.11,1.23,0.89,0.88,1.22,0.92,1.41,1.14,1.20,1.36,1.01,1.30,0.73,1.10,1.20,1.02
0.40,1.25,1.34,1.03,1.03,1.36,1.08,1.59,1.28,1.40,1.44,1.16,1.48,0.84,1.27,1.31,1.15
0.56,1.46,1.50,1.18,1.23,1.62,1.24,1.81,1.48,1.72,1.64,1.41,1.78,1.05,1.50,1.46,1.32
0.72,1.63,1.56,1.33,1.38,1.74,1.35,2.01,1.70,2.00,1.79,1.67,1.99,1.20,1.67,1.63,1.39
0.88,1.76,1.58,1.44,1.52,1.83,1.48,2.16,1.85,2.29,1.94,1.87,2.21,1.33,1.81,1.73,1.41
1.00,1.84,1.55,1.49,1.62,1.90,1.54,2.23,1.91,2.44,2.01,1.98,2.31,1.45,1.92,1.78,1.43


In [136]:
display(HTML(result['Mean per Joint Position Error'].loc[t].to_html(float_format=lambda x: '%.2f' % x)))

,average,walking,eating,smoking,discussion,directions,greeting,phoning,posing,purchases,sitting,sittingdown,takingphoto,waiting,walkingdog,walkingtogether
0.08,23.82,33.90,16.49,17.33,24.51,18.81,30.81,19.88,24.72,27.00,16.98,24.46,17.05,21.83,36.96,26.52
0.16,44.37,63.60,31.55,32.60,45.20,36.09,57.92,37.30,47.87,49.29,31.67,44.93,31.76,40.63,65.34,49.78
0.32,76.08,109.81,55.26,57.08,76.84,64.39,97.27,66.81,87.17,80.62,55.96,74.82,57.25,72.44,99.56,85.91
0.40,88.23,125.49,64.96,66.29,88.67,76.00,110.85,78.40,103.52,92.58,66.32,87.33,68.22,85.31,110.34,99.23
0.56,107.45,145.86,81.35,80.32,108.72,91.59,130.60,96.46,132.36,112.13,85.16,111.01,88.36,104.57,126.73,116.47
0.72,121.61,154.37,94.43,91.45,123.55,103.83,144.76,111.02,157.86,127.23,100.98,129.63,105.24,117.10,140.92,121.84
0.88,131.62,150.72,100.68,98.14,131.51,114.85,156.35,121.63,179.80,139.69,114.35,144.37,118.33,125.85,154.90,123.09
1.00,136.64,140.24,102.09,101.10,135.34,121.05,160.45,127.50,195.03,147.98,122.72,155.12,127.18,130.28,160.79,122.73
